In [109]:
import pandas as pd
import numpy as np
from helpers import *
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import MaxNLocator
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap
from IPython.display import display
from geopy.distance import geodesic
import math
from statsmodels.stats import diagnostic
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
beers = pd.read_csv("dataset/FilteredData/beers.csv")
breweries = pd.read_csv("dataset/FilteredData/breweries.csv")
ratings = pd.read_csv("dataset/FilteredData/ratings.csv", dtype = {'user_id': str})
users = pd.read_csv("dataset/FilteredData/users.csv", dtype = {'user_id': str})

In [111]:
all_years_list = sorted(ratings['year'].unique())
style_popularities_by_year = pd.DataFrame()
for year in all_years_list:
    location_style = location_style_stats(ratings.copy(), users.copy(), year, loc_style_threshold = 10, rating_flag = False)
    location_style_groupedby_loc = location_style.groupby('location')
    idx_most_popular = location_style_groupedby_loc['popularity_percentage'].idxmax()
    most_popular_styles_by_location = location_style.loc[idx_most_popular]
    most_popular_styles_by_location = most_popular_styles_by_location[['location', 'style']]
    most_popular_styles_by_location.set_index('location', inplace = True)
    most_popular_styles_by_location = most_popular_styles_by_location.rename(columns = {'style': str(year)})
    style_popularities_by_year = pd.concat([style_popularities_by_year, most_popular_styles_by_location], axis = 1)

style_popularities_by_year

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
location,,,,,,,,,,,,
Australia,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,American Pale Ale,American Pale Ale,American Pale Ale,American Pale Ale,American Pale Ale,American Pale Ale
Belgium,Belgian Strong Ale,Belgian Strong Ale,Belgian Strong Ale,Belgian Strong Ale,Belgian Strong Ale,Belgian Strong Ale,Belgian Strong Ale,Belgian Strong Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale
Brazil,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager
Canada,Fruit Beer,Cream Ale,Scotch Ale,Fruit Beer,Cream Ale,Scotch Ale,Spice/Herb/Vegetable,Saison,Fruit Beer,Saison,Saison,Sour/Wild Ale
Croatia,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pilsener,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager
...,...,...,...,...,...,...,...,...,...,...,...,...
"United States, Virginia",India Pale Ale (IPA),American Strong Ale,India Pale Ale (IPA),Black IPA,India Pale Ale (IPA),Spice/Herb/Vegetable,Spice/Herb/Vegetable,Spice/Herb/Vegetable,Spice/Herb/Vegetable,Spice/Herb/Vegetable,Sweet Stout,Imperial IPA
"United States, Washington",India Pale Ale (IPA),Amber Ale,India Pale Ale (IPA),Imperial IPA,American Strong Ale,American Strong Ale,Saison,Saison,Saison,Barley Wine,India Pale Ale (IPA),India Pale Ale (IPA)
"United States, West Virginia",India Pale Ale (IPA),India Pale Ale (IPA),Amber Ale,Porter,Amber Ale,Spice/Herb/Vegetable,Spice/Herb/Vegetable,Spice/Herb/Vegetable,Spice/Herb/Vegetable,Spice/Herb/Vegetable,Pilsener,German Hefeweizen


In [112]:
all_years_list = sorted(ratings['year'].unique())
style_ratings_by_year = pd.DataFrame()
for year in all_years_list:
    location_style = location_style_stats(ratings.copy(), users.copy(), year, loc_style_threshold = 10, rating_flag = True)
    location_style_groupedby_loc = location_style.groupby('location')
    idx_highest_rated = location_style_groupedby_loc['z_score'].idxmax()
    highest_rated_styles_by_location = location_style.loc[idx_highest_rated]
    highest_rated_styles_by_location = highest_rated_styles_by_location[['location', 'style']]
    highest_rated_styles_by_location.set_index('location', inplace = True)
    highest_rated_styles_by_location = highest_rated_styles_by_location.rename(columns = {'style': str(year)})
    style_ratings_by_year = pd.concat([style_ratings_by_year, highest_rated_styles_by_location], axis = 1)

style_ratings_by_year

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
location,,,,,,,,,,,,
Australia,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Eisbock,Abt/Quadrupel,Lambic Style - Gueuze,Imperial Stout,Abt/Quadrupel
Belgium,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Lambic Style - Gueuze,Abt/Quadrupel,Imperial Stout,American Strong Ale,Imperial Stout,Imperial Stout,Imperial Stout
Brazil,German Hefeweizen,Abt/Quadrupel,Abt/Quadrupel,Belgian Strong Ale,Imperial Stout,Lambic Style - Fruit,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abbey Tripel
Canada,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Lambic Style - Gueuze,Abt/Quadrupel,Abt/Quadrupel,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze
Croatia,Belgian Strong Ale,Porter,Abt/Quadrupel,Imperial Stout,Imperial Stout,Imperial Stout,Imperial Stout,Imperial Stout,Imperial Stout,Imperial Stout,Abt/Quadrupel,Lambic Style - Fruit
...,...,...,...,...,...,...,...,...,...,...,...,...
"United States, Virginia",Abt/Quadrupel,Abt/Quadrupel,Sour Red/Brown,Lambic Style - Gueuze,Sour/Wild Ale,Lambic Style - Gueuze,Lambic Style - Unblended,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze
"United States, Washington",Abt/Quadrupel,Abt/Quadrupel,Lambic Style - Gueuze,Sour Red/Brown,Lambic Style - Unblended,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze
"United States, West Virginia",Doppelbock,Imperial Stout,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Imperial Stout,Lambic Style - Fruit,Imperial Stout,Abt/Quadrupel


In [113]:
all_years_list = sorted(ratings['year'].unique())
bc_popularities_by_year = pd.DataFrame()
for year in all_years_list:
    location_bc = location_brewery_country_stats(ratings.copy(), users.copy(), breweries.copy(), year, loc_brewery_threshold = 20, rating_flag = False)
    location_bc_gb_loc = location_bc.groupby('location')
    idx_most_popular = location_bc_gb_loc['popularity_percentage'].idxmax()
    most_popular_bc_by_location = location_bc.loc[idx_most_popular]
    most_popular_bc_by_location = most_popular_bc_by_location[['location', 'brewery_location']]
    most_popular_bc_by_location.set_index('location', inplace = True)
    most_popular_bc_by_location = most_popular_bc_by_location.rename(columns = {'brewery_location': str(year)})
    bc_popularities_by_year = pd.concat([bc_popularities_by_year, most_popular_bc_by_location], axis = 1)
    
bc_popularities_by_year

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
location,,,,,,,,,,,,
Australia,Australia,Australia,Australia,Australia,Australia,Australia,Australia,Australia,Australia,Australia,Australia,Australia
Belgium,Belgium,Belgium,Belgium,Belgium,Belgium,Belgium,Belgium,Belgium,Belgium,Belgium,Belgium,Belgium
Brazil,Brazil,Belgium,Brazil,Brazil,Brazil,Brazil,Brazil,Brazil,Brazil,Brazil,Brazil,Brazil
Canada,Canada,Canada,Canada,Canada,Canada,Canada,Canada,Canada,Canada,Canada,Canada,Canada
Croatia,Austria,Croatia,Germany,Czech Republic,Germany,Germany,Germany,Germany,Germany,Germany,Croatia,Croatia
...,...,...,...,...,...,...,...,...,...,...,...,...
"United States, Virginia","United States, Virginia","United States, Virginia","United States, New York","United States, Virginia","United States, Virginia","United States, Virginia","United States, Virginia","United States, Virginia","United States, Virginia","United States, Virginia","United States, Virginia","United States, Virginia"
"United States, Washington","United States, Washington","United States, California","United States, California","United States, Washington","United States, Washington","United States, Oregon","United States, Washington","United States, Washington","United States, Washington","United States, Washington","United States, Washington","United States, Washington"
"United States, West Virginia","United States, Pennsylvania","United States, Pennsylvania","United States, New York","United States, New York","United States, New York","United States, Pennsylvania","United States, Pennsylvania","United States, Massachusetts","United States, Pennsylvania","United States, Ohio","United States, Ohio","United States, Ohio"


In [114]:
all_years_list = sorted(ratings['year'].unique())
bc_ratings_by_year = pd.DataFrame()
for year in all_years_list:
    location_bc = location_brewery_country_stats(ratings.copy(), users.copy(), breweries.copy(), year, loc_brewery_threshold = 20, rating_flag = True)
    location_bc_gb_loc = location_bc.groupby('location')
    idx_highest_rated = location_bc_gb_loc['z_score'].idxmax()
    highest_rated_bc_by_location = location_bc.loc[idx_highest_rated]
    highest_rated_bc_by_location = highest_rated_bc_by_location[['location', 'brewery_location']]
    highest_rated_bc_by_location.set_index('location', inplace=True)
    highest_rated_bc_by_location = highest_rated_bc_by_location.rename(columns={'brewery_location': str(year)})
    bc_ratings_by_year = pd.concat([bc_ratings_by_year, highest_rated_bc_by_location], axis=1)
    
bc_ratings_by_year

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
location,,,,,,,,,,,,
Australia,"United States, California","United States, Colorado","United States, Oregon",Norway,Norway,"United States, Michigan","United States, Alaska",Norway,"United States, Vermont",Norway,"United States, Oklahoma","United States, Massachusetts"
Belgium,"United States, California","United States, California","United States, Michigan","United States, Georgia","United States, Oregon",Norway,"United States, Illinois","United States, Vermont","United States, Iowa","United States, Vermont","United States, South Carolina","United States, Ohio"
Brazil,Germany,Canada,"United States, California",Belgium,Belgium,"United States, California",Belgium,"United States, Indiana","United States, Florida","United States, Illinois","United States, Florida","United States, Massachusetts"
Canada,"United States, Alaska",Norway,Norway,Norway,Norway,Norway,"United States, Indiana","United States, Iowa","United States, Vermont","United States, Oklahoma","United States, Iowa","United States, Oklahoma"
Croatia,Belgium,Belgium,Belgium,England,Scotland,Sweden,Belgium,Scotland,Norway,Norway,Norway,"United States, Oregon"
...,...,...,...,...,...,...,...,...,...,...,...,...
"United States, Virginia","United States, Connecticut","United States, Connecticut",Norway,Norway,"United States, Minnesota","United States, Alaska","United States, Alaska","United States, Iowa","United States, Iowa",Norway,Switzerland,"United States, Oklahoma"
"United States, Washington","United States, Connecticut",Norway,Norway,Norway,"United States, Indiana",Switzerland,Belgium,"United States, Iowa","United States, Iowa","United States, Vermont","United States, Iowa","United States, Wyoming"
"United States, West Virginia","United States, Ohio",Belgium,"United States, Michigan",Belgium,Belgium,Belgium,"United States, Michigan",Belgium,Belgium,"United States, Indiana",Belgium,"United States, Oregon"


GLOBAL TRENDS

In [115]:
all_years_list = sorted(ratings['year'].unique())
global_style_popularity_points_by_year = pd.DataFrame()
points = [12, 10, 8, 7, 6, 5, 4, 3, 2, 1]
for year in all_years_list:
    style_popularity_points_dict = {key: 0 for key in ratings['style'].unique().tolist()}
    location_style = location_style_stats(ratings.copy(), users.copy(), year, loc_style_threshold = 10, rating_flag = False)
    location_style_groupedby_loc = location_style.groupby('location')
    idx_10_most_popular = location_style_groupedby_loc['popularity_percentage'].nlargest(10).index.get_level_values(1)
    most_popular_styles_by_location = location_style.loc[idx_10_most_popular]
    
    point_idx = 0
    prev_location = most_popular_styles_by_location.iloc[0]['location']
    for index, row in most_popular_styles_by_location.iterrows():
        location = row['location']
        if location != prev_location:
            point_idx = 0
            prev_location = location
        style_popularity_points_dict[row['style']] += points[point_idx]
        point_idx += 1
    global_style_popularity_points_by_year[str(year)] = style_popularity_points_dict

global_style_popularities_by_year = global_style_popularity_points_by_year.apply(lambda x: x.nlargest(10).index.tolist()).T

global_style_popularities_by_year.columns = ['1st Most Popular', '2nd Most Popular', '3rd Most Popular', '4th Most Popular', '5th Most Popular', '6th Most Popular', '7th Most Popular', '8th Most Popular', '9th Most Popular', '10th Most Popular']
global_style_popularities_by_year.index.name = 'Year'
global_style_popularities_by_year

,1st Most Popular,2nd Most Popular,3rd Most Popular,4th Most Popular,5th Most Popular,6th Most Popular,7th Most Popular,8th Most Popular,9th Most Popular,10th Most Popular
Year,,,,,,,,,,
2006,India Pale Ale (IPA),Imperial Stout,Barley Wine,Pale Lager,Belgian Strong Ale,Spice/Herb/Vegetable,Pilsener,Porter,Brown Ale,Belgian Ale
2007,Imperial Stout,American Strong Ale,Pale Lager,Pilsener,Amber Ale,India Pale Ale (IPA),Imperial IPA,Saison,Amber Lager/Vienna,American Pale Ale
2008,Pale Lager,Imperial IPA,Sweet Stout,Imperial Stout,Belgian Strong Ale,India Pale Ale (IPA),Porter,Saison,Pilsener,Barley Wine
2009,Spice/Herb/Vegetable,Pilsener,India Pale Ale (IPA),Saison,Witbier,American Strong Ale,Imperial IPA,Barley Wine,Porter,Pale Lager
2010,Pale Lager,Imperial Stout,Spice/Herb/Vegetable,India Pale Ale (IPA),Imperial IPA,American Strong Ale,Belgian Strong Ale,Porter,Sour/Wild Ale,Barley Wine
2011,Spice/Herb/Vegetable,Imperial Stout,Pale Lager,Saison,Pilsener,Imperial IPA,Wheat Ale,Sour/Wild Ale,India Pale Ale (IPA),Amber Ale
2012,Spice/Herb/Vegetable,Pale Lager,Sour/Wild Ale,Barley Wine,Amber Lager/Vienna,Imperial IPA,Scotch Ale,Wheat Ale,Imperial Stout,Saison
2013,Sweet Stout,Imperial Stout,Brown Ale,Spice/Herb/Vegetable,Wheat Ale,Saison,Pale Lager,American Pale Ale,Berliner Weisse,Imperial IPA
2014,Imperial Stout,American Pale Ale,Imperial IPA,India Pale Ale (IPA),Wheat Ale,Spice/Herb/Vegetable,Pale Lager,Saison,Brown Ale,Fruit Beer


In [116]:
all_years_list = sorted(ratings['year'].unique())
global_style_rating_points_by_year = pd.DataFrame()
points = [12, 10, 8, 7, 6, 5, 4, 3, 2, 1]
for year in all_years_list:
    style_rating_points_dict = {key: 0 for key in ratings['style'].unique().tolist()}
    location_style = location_style_stats(ratings.copy(), users.copy(), year, loc_style_threshold = 10, rating_flag = True)
    location_style_groupedby_loc = location_style.groupby('location')
    idx_10_highest_rated = location_style_groupedby_loc['z_score'].nlargest(10).index.get_level_values(1)
    highest_rated_styles_by_location = location_style.loc[idx_10_highest_rated]
    
    point_idx = 0
    prev_location = highest_rated_styles_by_location.iloc[0]['location']
    for index, row in highest_rated_styles_by_location.iterrows():
        location = row['location']
        if location != prev_location:
            point_idx = 0
            prev_location = location
        style_rating_points_dict[row['style']] += points[point_idx]
        point_idx += 1
    global_style_rating_points_by_year[str(year)] = style_rating_points_dict

global_style_highest_rated_by_year = global_style_rating_points_by_year.apply(lambda x: x.nlargest(10).index.tolist()).T

global_style_highest_rated_by_year.columns = ['1st Highest Rated', '2nd Highest Rated', '3rd Highest Rated', '4th Highest Rated', '5th Highest Rated', '6th Highest Rated', '7th Highest Rated', '8th Highest Rated', '9th Highest Rated', '10th Highest Rated']
global_style_highest_rated_by_year.index.name = 'Year'
global_style_highest_rated_by_year

,1st Highest Rated,2nd Highest Rated,3rd Highest Rated,4th Highest Rated,5th Highest Rated,6th Highest Rated,7th Highest Rated,8th Highest Rated,9th Highest Rated,10th Highest Rated
Year,,,,,,,,,,
2006,Imperial Stout,Abt/Quadrupel,Imperial IPA,Belgian Strong Ale,Sour/Wild Ale,Eisbock,Lambic Style - Gueuze,Sour Red/Brown,Baltic Porter,Barley Wine
2007,Abt/Quadrupel,Imperial Stout,Imperial IPA,Sour/Wild Ale,Belgian Strong Ale,Sour Red/Brown,Baltic Porter,Lambic Style - Gueuze,Barley Wine,American Strong Ale
2008,Abt/Quadrupel,Imperial Stout,Imperial IPA,Lambic Style - Gueuze,Sour/Wild Ale,Sour Red/Brown,Belgian Strong Ale,Eisbock,Barley Wine,Baltic Porter
2009,Imperial Stout,Abt/Quadrupel,Sour Red/Brown,Imperial IPA,Lambic Style - Gueuze,Sour/Wild Ale,Barley Wine,Belgian Strong Ale,Eisbock,American Strong Ale
2010,Imperial Stout,Abt/Quadrupel,Lambic Style - Gueuze,Sour/Wild Ale,Sour Red/Brown,Imperial IPA,Lambic Style - Unblended,Weizen Bock,American Strong Ale,Belgian Strong Ale
2011,Imperial Stout,Abt/Quadrupel,Lambic Style - Gueuze,Imperial IPA,Sour/Wild Ale,Lambic Style - Unblended,Lambic Style - Fruit,Sour Red/Brown,Old Ale,Barley Wine
2012,Imperial Stout,Abt/Quadrupel,Lambic Style - Gueuze,Imperial IPA,Sour/Wild Ale,Sour Red/Brown,Barley Wine,Eisbock,Lambic Style - Fruit,Lambic Style - Unblended
2013,Imperial Stout,Lambic Style - Gueuze,Abt/Quadrupel,Imperial IPA,Sour/Wild Ale,Lambic Style - Fruit,Eisbock,Sour Red/Brown,Lambic Style - Unblended,Old Ale
2014,Imperial Stout,Lambic Style - Gueuze,Abt/Quadrupel,Sour/Wild Ale,Lambic Style - Fruit,Imperial IPA,Sour Red/Brown,Eisbock,Lambic Style - Unblended,Barley Wine


In [117]:
all_years_list = sorted(ratings['year'].unique())
global_brewery_popularity_points_by_year = pd.DataFrame()
points = [12, 10, 8, 7, 6, 5, 4, 3, 2, 1]
for year in all_years_list:
    brewery_popularity_points_dict = {key: 0 for key in breweries['location'].unique().tolist()}
    location_bc = location_brewery_country_stats(ratings.copy(), users.copy(), breweries.copy(), year, loc_brewery_threshold = 20, rating_flag = False)
    location_bc_groupedby_loc = location_bc.groupby('location')
    idx_10_most_popular = location_bc_groupedby_loc['popularity_percentage'].nlargest(10).index.get_level_values(1)
    most_popular_breweries_by_location = location_bc.loc[idx_10_most_popular]
    
    point_idx = 0
    prev_location = most_popular_breweries_by_location.iloc[0]['location']
    for index, row in most_popular_breweries_by_location.iterrows():
        location = row['location']
        if location != prev_location:
            point_idx = 0
            prev_location = location
        brewery_popularity_points_dict[row['brewery_location']] += points[point_idx]
        point_idx += 1
    global_brewery_popularity_points_by_year[str(year)] = brewery_popularity_points_dict

global_brewery_popularities_by_year = global_brewery_popularity_points_by_year.apply(lambda x: x.nlargest(10).index.tolist()).T

global_brewery_popularities_by_year.columns = ['1st Most Popular', '2nd Most Popular', '3rd Most Popular', '4th Most Popular', '5th Most Popular', '6th Most Popular', '7th Most Popular', '8th Most Popular', '9th Most Popular', '10th Most Popular']
global_brewery_popularities_by_year.index.name = 'Year'
global_brewery_popularities_by_year

,1st Most Popular,2nd Most Popular,3rd Most Popular,4th Most Popular,5th Most Popular,6th Most Popular,7th Most Popular,8th Most Popular,9th Most Popular,10th Most Popular
Year,,,,,,,,,,
2006,Belgium,"United States, New York","United States, California",Germany,"United States, Michigan","United States, Pennsylvania","United States, Colorado","United States, Delaware","United States, Wisconsin","United States, Massachusetts"
2007,"United States, Maryland","United States, Missouri","United States, Massachusetts","United States, Colorado","United States, New York","United States, Oregon","United States, Georgia","United States, Michigan","United States, California",England
2008,"United States, New York","United States, Missouri","United States, Massachusetts","United States, Colorado",Belgium,"United States, California","United States, Oregon",England,"United States, Maryland","United States, Pennsylvania"
2009,"United States, New York","United States, Colorado","United States, Missouri",Germany,"United States, Massachusetts","United States, Georgia","United States, Oregon","United States, Pennsylvania","United States, California",Belgium
2010,"United States, New York","United States, Colorado","United States, Massachusetts","United States, Oregon","United States, Georgia",Germany,England,Mexico,"United States, Missouri","United States, Michigan"
2011,"United States, New York","United States, Massachusetts","United States, Colorado","United States, Missouri","United States, Pennsylvania",Mexico,"United States, Georgia","United States, Illinois","United States, California",England
2012,Belgium,"United States, New York","United States, Colorado",Mexico,Germany,"United States, Missouri","United States, Oregon","United States, Massachusetts","United States, Georgia",Switzerland
2013,"United States, Massachusetts",Belgium,"United States, New York",Germany,"United States, Missouri",Mexico,"United States, California","United States, Colorado","United States, Georgia","United States, Pennsylvania"
2014,Belgium,Mexico,"United States, Colorado",Japan,Jamaica,"United States, Massachusetts","United States, Oregon",Germany,"United States, California","United States, Pennsylvania"


In [118]:
all_years_list = sorted(ratings['year'].unique())
global_brewery_rating_points_by_year = pd.DataFrame()
points = [12, 10, 8, 7, 6, 5, 4, 3, 2, 1]
for year in all_years_list:
    brewery_rating_points_dict = {key: 0 for key in breweries['location'].unique().tolist()}
    location_bc = location_brewery_country_stats(ratings.copy(), users.copy(), breweries.copy(), year, loc_brewery_threshold = 20, rating_flag = True)
    location_bc_groupedby_loc = location_bc.groupby('location')
    idx_10_highest_rated = location_bc_groupedby_loc['z_score'].nlargest(10).index.get_level_values(1)
    highest_rated_breweries_by_location = location_bc.loc[idx_10_highest_rated]
    
    point_idx = 0
    prev_location = highest_rated_breweries_by_location.iloc[0]['location']
    for index, row in highest_rated_breweries_by_location.iterrows():
        location = row['location']
        if location != prev_location:
            point_idx = 0
            prev_location = location
        brewery_rating_points_dict[row['brewery_location']] += points[point_idx]
        point_idx += 1
    global_brewery_rating_points_by_year[str(year)] = brewery_rating_points_dict

global_brewery_highest_rated_by_year = global_brewery_rating_points_by_year.apply(lambda x: x.nlargest(10).index.tolist()).T

global_brewery_highest_rated_by_year.columns = ['1st Highest Rated', '2nd Highest Rated', '3rd Highest Rated', '4th Highest Rated', '5th Highest Rated', '6th Highest Rated', '7th Highest Rated', '8th Highest Rated', '9th Highest Rated', '10th Highest Rated']
global_brewery_highest_rated_by_year.index.name = 'Year'
global_brewery_highest_rated_by_year

,1st Highest Rated,2nd Highest Rated,3rd Highest Rated,4th Highest Rated,5th Highest Rated,6th Highest Rated,7th Highest Rated,8th Highest Rated,9th Highest Rated,10th Highest Rated
Year,,,,,,,,,,
2006,Belgium,"United States, California","United States, Oregon","United States, Ohio",Germany,England,"United States, Michigan","United States, Indiana","United States, Alaska","United States, Colorado"
2007,Belgium,"United States, California",Norway,"United States, Ohio","United States, Oregon",Germany,England,"United States, Alaska","United States, Michigan",Canada
2008,Belgium,"United States, California",Norway,"United States, Oregon","United States, Ohio","United States, Alaska","United States, Michigan",Canada,England,"United States, New York"
2009,Belgium,"United States, California",Norway,"United States, Oregon","United States, Michigan","United States, Alaska","United States, Ohio",England,"United States, New York","United States, Illinois"
2010,Belgium,Norway,"United States, California","United States, Illinois","United States, Oregon","United States, Ohio","United States, Michigan","United States, Alaska","United States, Minnesota",Canada
2011,Belgium,"United States, California",Norway,"United States, Michigan","United States, Alaska","United States, Indiana","United States, Minnesota","United States, Illinois","United States, Ohio","United States, Oregon"
2012,Belgium,"United States, California","United States, Michigan",Norway,"United States, Indiana","United States, Alaska","United States, Ohio","United States, Illinois","United States, Vermont","United States, Minnesota"
2013,Belgium,"United States, Indiana",Norway,"United States, California","United States, Oklahoma","United States, Vermont","United States, Michigan","United States, Alaska","United States, Ohio","United States, Illinois"
2014,"United States, Iowa",Belgium,"United States, Oklahoma","United States, Vermont",Norway,"United States, Indiana","United States, Ohio","United States, Illinois","United States, South Carolina","United States, California"
